<a href="https://colab.research.google.com/github/irawan09/Click_Prediction/blob/master/Techinical_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Gaussian Naive Bayes

##Importing library

In [16]:
import pandas as pd
from math import sqrt
from math import pi
from math import exp

##Read dataset

###Read Train dataset

Read the dataset using pandas library and display 5 first row data

In [2]:
train_url = "https://media.githubusercontent.com/media/irawan09/Click_Prediction/master/train_set.csv"
traindataset=pd.read_csv(train_url)
traindataset.head()

,Session Id,DateTime,User Id,Product Type,Campaign Id,Webpage Id,Product Category,Advertisment Size,User Depth,Internet Browser Id,Gender,Age Group,City Size,Device Used,Clicked,Interested In Cars,Interested In Food,Interested In News,Interested In Technology,Interested In Medicine,Interested In Politics,Interested In Fashion,Interested In Astronomy,Interested In Animals,Interested In Travel
0,229ac4c2-0ee9-4a3b-b52c-3b20c9d43039,09-04-2021 00:00,858557,C,359520,13787,4,NaN,3.0,10.0,F,45-54,5900000.0,Mobile,No,1,0,1,0,1,0,0,0,1,0
1,87c0f74a-fa7a-4b3f-bc48-ad1a5f80de2e,09-04-2021 00:00,243253,C,105960,11085,5,NaN,2.0,8.0,F,25-34,NaN,Mobile,No,1,0,1,1,1,0,0,0,1,1
2,b746f0ad-1aa3-492c-b7fa-5dd95643fb51,09-04-2021 00:00,243253,C,359520,13787,4,NaN,2.0,8.0,F,25-34,NaN,Mobile,No,1,0,0,1,0,0,1,0,0,0
3,36257832-77d5-4d8d-9abd-25d49dffb20d,09-04-2021 00:00,1097446,I,359520,13787,3,NaN,3.0,3.0,M,35-44,500000.0,PC/Laptop,No,0,1,1,1,0,0,1,0,0,0
4,df3a66aa-4714-407d-b6fe-49fe03ce8f16,09-04-2021 00:01,663656,C,405490,60305,3,NaN,3.0,2.0,M,25-34,500000.0,PC/Laptop,No,0,1,0,0,0,1,1,1,0,0


In [3]:
traindataset.size

9795625

###Read Test dataset

In [4]:
test_url = "https://media.githubusercontent.com/media/irawan09/Click_Prediction/master/test_set.csv"
testdataset=pd.read_csv(test_url)
testdataset.head()

,Session Id,DateTime,User Id,Product Type,Campaign Id,Webpage Id,Product Category,Advertisment Size,User Depth,Internet Browser Id,Gender,Age Group,City Size,Device Used,Interested In Cars,Interested In Food,Interested In News,Interested In Technology,Interested In Medicine,Interested In Politics,Interested In Fashion,Interested In Astronomy,Interested In Animals,Interested In Travel
0,41cb91c7-ec1d-4e71-98fd-9e1c374bd33b,14-04-2021 00:00,262150,C,405490,60305,3,NaN,3.0,2.0,M,25-34,500000.0,PC/Laptop,0,1,0,1,0,1,1,1,0,1
1,c9d3f8e4-1445-45f2-99db-754bda50766c,14-04-2021 00:00,308173,C,405490,60305,3,NaN,3.0,10.0,F,45-54,500000.0,PC/Laptop,1,1,1,1,0,1,0,0,1,0
2,f85ffa39-b799-49a1-82b5-2f76a4009212,14-04-2021 00:00,308173,B,118601,28529,4,82527.0,3.0,10.0,F,45-54,500000.0,PC/Laptop,1,0,0,0,1,1,0,1,0,0
3,49336ffb-a9b7-485c-9891-6274482484e0,14-04-2021 00:00,806073,H,118601,28529,5,82527.0,3.0,3.0,M,35-44,500000.0,Mobile,1,0,1,0,1,1,0,1,1,1
4,8ea9c920-6686-46f2-acd1-4bf20371c28c,14-04-2021 00:00,796787,E,404347,53587,1,146115.0,1.0,3.0,M,35-44,NaN,Mobile,1,0,1,0,1,1,1,0,1,0


In [5]:
testdataset.size

1715184

##Data Preprocesing

###Preprocessing Train dataset

In [6]:
print (traindataset.groupby('Product Type').groups.keys())
pt_map = {"J" : 9,"I" : 8,"H" : 7,"G" : 6,"F" : 5,"E" : 4,"D" : 3,"C" : 2,"B" : 1, "A" : 0}
traindataset["Product Type"] = traindataset["Product Type"].map(pt_map)

print (traindataset.groupby('Gender').groups.keys())
s_map = {"M" : 1, "F" : 0}
traindataset["Gender"] = traindataset["Gender"].map(s_map)

print (traindataset.groupby('Age Group').groups.keys())
g_map = {">65" : 6, "<18" : 5, "55-64" : 4, "45-54" : 3, "35-44" : 2, "25-34" : 1, "18-24" : 0}
traindataset["Age Group"] = traindataset["Age Group"].map(g_map)


print (traindataset.groupby('Device Used').groups.keys())
du_map = {"Mobile" : 1, "PC/Laptop" : 0}
traindataset["Device Used"] = traindataset["Device Used"].map(du_map)

print (traindataset.groupby('Clicked').groups.keys())
c_map = {"Yes" : 1, "No" : 0}
traindataset["Clicked"] = traindataset["Clicked"].map(c_map)


dict_keys(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'])
dict_keys(['F', 'M'])
dict_keys(['18-24', '25-34', '35-44', '45-54', '55-64', '<18', '>65'])
dict_keys(['Mobile', 'PC/Laptop'])
dict_keys(['No', 'Yes'])


###Preprocessing Test dataset

In [7]:
print (testdataset.groupby('Product Type').groups.keys())
pt_map = {"J" : 9,"I" : 8,"H" : 7,"G" : 6,"F" : 5,"E" : 4,"D" : 3,"C" : 2,"B" : 1, "A" : 0}
testdataset["Product Type"] = testdataset["Product Type"].map(pt_map)

print (testdataset.groupby('Gender').groups.keys())
s_map = {"M" : 1, "F" : 0}
testdataset["Gender"] = testdataset["Gender"].map(s_map)

print (testdataset.groupby('Age Group').groups.keys())
g_map = {">65" : 6, "<18" : 5, "55-64" : 4, "45-54" : 3, "35-44" : 2, "25-34" : 1, "18-24" : 0}
testdataset["Age Group"] = testdataset["Age Group"].map(g_map)


print (testdataset.groupby('Device Used').groups.keys())
du_map = {"Mobile" : 1, "PC/Laptop" : 0}
testdataset["Device Used"] = testdataset["Device Used"].map(du_map)

dict_keys(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'])
dict_keys(['F', 'M'])
dict_keys(['18-24', '25-34', '35-44', '45-54', '55-64', '<18', '>65'])
dict_keys(['Mobile', 'PC/Laptop'])


##Deal with missing data

In [8]:
print(traindataset.isnull().sum())

Session Id                       0
DateTime                         0
User Id                          0
Product Type                     0
Campaign Id                      0
Webpage Id                       0
Product Category                 0
Advertisment Size           321591
User Depth                   15743
Internet Browser Id          15743
Gender                       15743
Age Group                    15743
City Size                   107129
Device Used                      0
Clicked                          0
Interested In Cars               0
Interested In Food               0
Interested In News               0
Interested In Technology         0
Interested In Medicine           0
Interested In Politics           0
Interested In Fashion            0
Interested In Astronomy          0
Interested In Animals            0
Interested In Travel             0
dtype: int64


In [9]:
print(testdataset.isnull().sum())

Session Id                      0
DateTime                        0
User Id                         0
Product Type                    0
Campaign Id                     0
Webpage Id                      0
Product Category                0
Advertisment Size           44263
User Depth                   2500
Internet Browser Id          2500
Gender                       2500
Age Group                    2500
City Size                   18000
Device Used                     0
Interested In Cars              0
Interested In Food              0
Interested In News              0
Interested In Technology        0
Interested In Medicine          0
Interested In Politics          0
Interested In Fashion           0
Interested In Astronomy         0
Interested In Animals           0
Interested In Travel            0
dtype: int64


In [10]:
train_dataset = traindataset.drop(['Webpage Id','Campaign Id','User Id', 'DateTime', 'Advertisment Size', 'User Depth', 'Internet Browser Id', 'Gender', 'Age Group', 'City Size', ], axis = 1)


In [11]:
test_dataset = testdataset.drop(['Webpage Id','Campaign Id','User Id','DateTime', 'Advertisment Size', 'User Depth', 'Internet Browser Id', 'Gender', 'Age Group', 'City Size', ], axis = 1)

In [14]:
print(train_dataset.isnull().sum())

Session Id                  0
Product Type                0
Product Category            0
Device Used                 0
Clicked                     0
Interested In Cars          0
Interested In Food          0
Interested In News          0
Interested In Technology    0
Interested In Medicine      0
Interested In Politics      0
Interested In Fashion       0
Interested In Astronomy     0
Interested In Animals       0
Interested In Travel        0
dtype: int64


In [15]:
print(test_dataset.isnull().sum())

Session Id                  0
Product Type                0
Product Category            0
Device Used                 0
Interested In Cars          0
Interested In Food          0
Interested In News          0
Interested In Technology    0
Interested In Medicine      0
Interested In Politics      0
Interested In Fashion       0
Interested In Astronomy     0
Interested In Animals       0
Interested In Travel        0
dtype: int64


##Pre Analysis Model